# 练习三 动态规划进行策略评估、策略迭代和价值迭代

## 1 小型方格世界MDP建模

In [1]:
S = [i for i in range(16)] # 状态空间
A = ["n", "e", "s", "w"] # 行为空间
# P,R,将由dynamics动态生成

ds_actions = {"n": -4, "e": 1, "s": 4, "w": -1} # 行为对状态的改变

def dynamics(s, a): # 环境动力学
    '''模拟小型方格世界的环境动力学特征
    Args:
        s 当前状态 int 0 - 15
        a 行为 str in ['n','e','s','w'] 分别表示北、东、南、西
    Returns: tuple (s_prime, reward, is_end)
        s_prime 后续状态
        reward 奖励值
        is_end 是否进入终止状态

    将s_prime首先初始化为s，方便进行状态更新。

    首先判断是否碰到了边界，如果碰到边界则说明状态不能采取该动作。
    如果没有碰到边界，则新的状态 = 旧的状态 + 行为改变的大小。
    例如，旧的状态是状态2， 采取的行为是's'，那么就有:
    s_prime = 2 + 4  (4是行为s对应的字典值)
    那么s_prime就是6，正好对应了地图中的2向南移会到6.

    如果当前状态是终止状态，那么行为获得的奖励就是0.否则是-1.
    这里值得注意的是，如果当前状态是终止状态，前面的if else中的s_prime就不会
    被更新了，也就是说到了终止状态，状态就不会再改变了。此时无论行为是什么，都不会采取
    实际的行动，所以获得的奖励始终为0.

    判断是否为终止状态，是的话is_end就是True，否则就是false。
    返回:后续状态、行为的奖励、是否进入终止状态
    '''
    s_prime = s
    if (s%4 == 0 and a == "w") or (s<4 and a == "n") \
        or ((s+1)%4 == 0 and a == "e") or (s > 11 and a == "s")\
        or s in [0, 15]:
        pass
    else:
        ds = ds_actions[a]
        s_prime = s + ds
    reward = 0 if s in [0, 15] else -1
    is_end = True if s in [0, 15] else False
    return s_prime, reward, is_end

def P(s, a, s1): # 状态转移概率函数
    '''
    根据环境动力学函数来计算转移概率
    注意如果 dynamics(s, a) 中的状态s采取的行为a是可行的话（比如在状态5采取行为向
    四个方向移动都是可行的），那么就一定会移动到新的状态且获得一个-1的奖励。
    但是这里只计算概率，不关心奖励，所以用 _ 来占位表示得到了返回值但不使用。

    返回s1 == s_prime是因为，在强化学习问题中一个状态s采取一个行为a能够转移到的
    新状态可能是多个，比如我在学习中采取浏览手机的行为，可能会导致我接着学习状态，
    也可能转移到继续浏览手机这样的状态。
    而在这个迷宫中，我们在某个状态s采取行为a前往新的状态的概率是固定的。所以只有
    在s1 == s_prime 时返回概率1， 其他都会返回0.
    '''
    s_prime, _, _ = dynamics(s, a)
    return s1 == s_prime

def R(s, a): # 奖励函数
    '''
    根据环境动力学函数得到某状态下采取行为的奖励.

    只有两种可能，状态s非终止状态则返回-1，否则返回0.
    所以其实这个函数可以改写成

    return 0 if s in [0,15] else -1
    '''
    _, r, _ = dynamics(s, a)
    return r

gamma = 1.00
MDP = S, A, R, P, gamma  #这里的R和P都是函数，将它们传给MDP实际上是传递了函数的地址。

## 2 策略

In [2]:
def uniform_random_pi(MDP = None, V = None, s = None, a = None):
    '''
    均一随机策略,在任何状态下朝着能够前进的方向前进的概率均等。
    '''
    _, A, _, _, _ = MDP
    n = len(A)
    return 0 if n == 0 else 1.0/n

def greedy_pi(MDP, V, s, a):
    '''
    贪心算法进行策略迭代，参数分别是马尔科夫决策过程的五个参数S,A,R,P,gamma
    V 代表当前的状态价值，s和a分别表示状态和动作。
    注：策略的更新是基于状态价值的，也就是说每一轮更新得到的策略都是基于当前
    状态价值的最优策略。
    某一状态下的最优策略，则是从该状态下所有可能的行为中采取行为价值最高的行为。
    该行为可能不止一个。

    首先将最大的状态价值初始化为无限小， 达到最大的状态价值对应的行为空间为空。

    然后统计在当前状态下
    '''
    S, A, P, R, gamma = MDP
    max_v, a_max_v = -float('inf'), []
    for a_opt in A:# 统计后续状态的最大价值以及到达到达该状态的行为（可能不止一个）
        s_prime, reward, _ = dynamics(s, a_opt)  #状态s采取行为a_opt后得到奖励并到达新的状态
        v_s_prime = get_value(V, s_prime)   # 得到新的状态的状态价值
        if v_s_prime > max_v:               # 如果新的状态的状态价值大于之前所找到的最大的状态价值
            max_v = v_s_prime  #说明该行为是比之前的行为更优的，更新最大的后续最大状态价值
            a_max_v = [a_opt]   # 将达到最大状态价值对应的行为设置为空
        elif(v_s_prime == max_v):   # 达到最大状态价值的行为可能不止一个
            a_max_v.append(a_opt)
    n = len(a_max_v)
    if n == 0: return 0.0
    return 1.0/n if a in a_max_v else 0.0       # 更新策略

def get_pi(Pi, s, a, MDP = None, V = None):
    '''
    这里的参数pi是一个具体的策略，该函数根据策略返回状态s下采取行为a的概率
    '''
    return Pi(MDP, V, s, a)

In [3]:
# 辅助函数
def get_prob(P, s, a, s1): # 获取状态转移概率
    return P(s, a, s1)

def get_reward(R, s, a): # 获取奖励值
    return R(s, a)

def set_value(V, s, v): # 设置价值字典
    V[s] = v
    
def get_value(V, s): # 获取状态价值
    return V[s]

def display_V(V): # 显示状态价值
    for i in range(16):
        print('{0:>6.2f}'.format(V[i]),end = " ")
        if (i+1) % 4 == 0:
            print("")
    print()

## 3. 策略评估

In [4]:
def compute_q(MDP, V, s, a):
    '''根据给定的MDP，价值函数V，计算状态行为对s,a的价值qsa
    '''
    S, A, R, P, gamma = MDP
    q_sa = 0
    for s_prime in S:
        q_sa += get_prob(P, s, a, s_prime) * get_value(V, s_prime)
    q_sa = get_reward(R, s,a) + gamma * q_sa
    return q_sa

def compute_v(MDP, V, Pi, s):
    '''给定MDP下依据某一策略Pi和当前状态价值函数V计算某状态s的价值
    '''
    S, A, R, P, gamma = MDP
    v_s = 0
    for a in A:
        v_s += get_pi(Pi, s, a, MDP, V) * compute_q(MDP, V, s, a)
    return v_s        

def update_V(MDP, V, Pi):
    '''给定一个MDP和一个策略，更新该策略下的价值函数V
    该函数每次都在一轮策略评估完成后调用，更新该策略下每个状态的
    状态价值
    '''
    S, _, _, _, _ = MDP
    V_prime = V.copy()
    for s in S:
        set_value(V_prime, s, compute_v(MDP, V_prime, Pi, s))
    return V_prime


def policy_evaluate(MDP, V, Pi, n):
    '''使用n次迭代计算来评估一个MDP在给定策略Pi下的状态价值，初始时价值为V
    '''
    for i in range(n):
        #print("====第{}次迭代====".format(i+1))
        V = update_V(MDP, V, Pi)
        #display_V(V)
    return V


In [5]:
V = [0  for _ in range(16)] # 状态价值
V_pi = policy_evaluate(MDP, V, uniform_random_pi, 100)
display_V(V_pi)

V = [0  for _ in range(16)] # 状态价值
V_pi = policy_evaluate(MDP, V, greedy_pi, 100)
display_V(V_pi)

  0.00 -14.00 -20.00 -22.00 
-14.00 -18.00 -20.00 -20.00 
-20.00 -20.00 -18.00 -14.00 
-22.00 -20.00 -14.00   0.00 

  0.00  -1.00  -2.00  -3.00 
 -1.00  -2.00  -3.00  -2.00 
 -2.00  -3.00  -2.00  -1.00 
 -3.00  -2.00  -1.00   0.00 



## 4 策略迭代

In [6]:
def policy_iterate(MDP, V, Pi, n, m):
    for i in range(m):
        V = policy_evaluate(MDP, V, Pi, n)
        Pi = greedy_pi # 第一次迭代产生新的价值函数后随机使用贪婪策略
    return V

In [7]:
V = [0  for _ in range(16)] # 重置状态价值
V_pi = policy_iterate(MDP, V, greedy_pi, 1, 100)
display_V(V_pi)

  0.00  -1.00  -2.00  -3.00 
 -1.00  -2.00  -3.00  -2.00 
 -2.00  -3.00  -2.00  -1.00 
 -3.00  -2.00  -1.00   0.00 



## 价值迭代

In [8]:
# 价值迭代得到最优状态价值过程
def compute_v_from_max_q(MDP, V, s):
    '''根据一个状态的下所有可能的行为价值中最大一个来确定当前状态价值
    行为价值由行为获得的即时奖励和后续状态价值的期望获得。
    选择了最大的行为价值意味着获得了最大的后续状态价值。
    '''
    S, A, R, P, gamma = MDP
    v_s = -float('inf')
    for a in A:
        qsa = compute_q(MDP, V, s, a)
        if qsa >= v_s:
            v_s = qsa
    return v_s

def update_V_without_pi(MDP, V):
    '''在不依赖策略的情况下直接通过后续状态的价值来更新状态价值
    通过遍历所有状态下所有可能的行为价值，得到最大的作为状态价值
    进行更新。
    '''
    S, _, _, _, _ = MDP
    V_prime = V.copy()
    for s in S:
        set_value(V_prime, s, compute_v_from_max_q(MDP, V_prime, s))
    return V_prime

def value_iterate(MDP, V, n):
    '''价值迭代
    '''
    for i in range(n):
        V = update_V_without_pi(MDP, V)
        display_V(V)
    return V

In [9]:
V = [0  for _ in range(16)] # 重置状态价值
display_V(V)

V_star = value_iterate(MDP, V, 4)
display_V(V_star)

  0.00   0.00   0.00   0.00 
  0.00   0.00   0.00   0.00 
  0.00   0.00   0.00   0.00 
  0.00   0.00   0.00   0.00 

  0.00  -1.00  -1.00  -1.00 
 -1.00  -1.00  -1.00  -1.00 
 -1.00  -1.00  -1.00  -1.00 
 -1.00  -1.00  -1.00   0.00 

  0.00  -1.00  -2.00  -2.00 
 -1.00  -2.00  -2.00  -2.00 
 -2.00  -2.00  -2.00  -1.00 
 -2.00  -2.00  -1.00   0.00 

  0.00  -1.00  -2.00  -3.00 
 -1.00  -2.00  -3.00  -2.00 
 -2.00  -3.00  -2.00  -1.00 
 -3.00  -2.00  -1.00   0.00 

  0.00  -1.00  -2.00  -3.00 
 -1.00  -2.00  -3.00  -2.00 
 -2.00  -3.00  -2.00  -1.00 
 -3.00  -2.00  -1.00   0.00 

  0.00  -1.00  -2.00  -3.00 
 -1.00  -2.00  -3.00  -2.00 
 -2.00  -3.00  -2.00  -1.00 
 -3.00  -2.00  -1.00   0.00 



In [10]:
def greedy_policy(MDP, V, s):
    '''
    贪心策略的字符串版本
    '''
    S, A, P, R, gamma = MDP
    max_v, a_max_v = -float('inf'), []
    for a_opt in A:# 统计后续状态的最大价值以及到达到达该状态的行为（可能不止一个）
        s_prime, reward, _ = dynamics(s, a_opt)
        v_s_prime = get_value(V, s_prime)
        if v_s_prime > max_v:
            max_v = v_s_prime
            a_max_v = a_opt
        elif(v_s_prime == max_v):
            a_max_v += a_opt
    return str(a_max_v)

def display_policy(policy, MDP, V):
    S, A, P, R, gamma = MDP
    for i in range(16):
        print('{0:^6}'.format(policy(MDP, V, S[i])),end = " ")
        if (i+1) % 4 == 0:
            print("")
    print()

In [11]:
display_policy(greedy_policy, MDP, V_star)

 nesw    w      w      sw   
  n      nw    nesw    s    
  n     nesw    es     s    
  ne     e      e     nesw  

